Изложение построено по [этому](https://www.youtube.com/watch?v=kCc8FmEb1nY&list=PLAqhIrjkxbuWI23v9cThsA9GvCAUhRvKZ&index=7) материалу, спасибо Andrej Karpathy за то, что поделился своим опытом с миром

In [36]:
import torch
import torch.nn as nn
from torch.nn import functional as F

Попытаемся за разумное время построить модель, которая будет генерировать связный текст.

## Составим словарь
В этот раз будем работать на уровне отдельных символов

In [37]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()
print(f"Всего {len(text)} символа\n------------\n")
print(text[:100])

Всего 1115394 символа
------------

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [38]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [39]:
print(f'Нулевой элемент нашего словаря - это {repr(chars[0])}')

Нулевой элемент нашего словаря - это '\n'


In [40]:
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(encode("hi there"))
print(decode(encode("hi there")))

[46, 47, 1, 58, 46, 43, 56, 43]
hi there


Раз у нас на повестке дня нейросети, будем использовать Pytorch. Превратим наши данные в подходящий ему формат:

In [41]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:100])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [42]:
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

Для начала будем оперировать контекcтом размера 8:

In [43]:
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [44]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f'Для инпута "{context}" целевой символ "{target}"')

Для инпута "tensor([18])" целевой символ "47"
Для инпута "tensor([18, 47])" целевой символ "56"
Для инпута "tensor([18, 47, 56])" целевой символ "57"
Для инпута "tensor([18, 47, 56, 57])" целевой символ "58"
Для инпута "tensor([18, 47, 56, 57, 58])" целевой символ "1"
Для инпута "tensor([18, 47, 56, 57, 58,  1])" целевой символ "15"
Для инпута "tensor([18, 47, 56, 57, 58,  1, 15])" целевой символ "47"
Для инпута "tensor([18, 47, 56, 57, 58,  1, 15, 47])" целевой символ "58"


In [45]:
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

def get_batch(split, block_size=8):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('Инпут:')
print(xb.shape)
print(xb)
print('Целевые символы:')
print(yb.shape)
print(yb)

print('----')

for t in range(block_size): # time dimension
    context = xb[0, :t+1]
    target = yb[0,t]
    print(f'Для инпута "{context.tolist()}" целевой символ "{target}"')

Инпут:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
Целевые символы:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
----
Для инпута "[24]" целевой символ "43"
Для инпута "[24, 43]" целевой символ "58"
Для инпута "[24, 43, 58]" целевой символ "5"
Для инпута "[24, 43, 58, 5]" целевой символ "57"
Для инпута "[24, 43, 58, 5, 57]" целевой символ "1"
Для инпута "[24, 43, 58, 5, 57, 1]" целевой символ "46"
Для инпута "[24, 43, 58, 5, 57, 1, 46]" целевой символ "43"
Для инпута "[24, 43, 58, 5, 57, 1, 46, 43]" целевой символ "39"


## Простая языковая модель

Которая работает по тому же принципу, что и word2vec, который мы рассматривали ранее. Для каждого элемента нашего словаря обучаем вектор, а потом используем эти векторы. Только сейчас элементы нашего словаря - символы, а не слова

In [46]:
class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
#!          смотрим только на предыдущий символ!
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
#!          берём не какой-то один самый вероятный символ, а сэмплируем случайный, используя полученные вероятности.
#!          в языке работает также есть [чёрный к...]от, а есть [чёрный к...]амаз
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

In [47]:
m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))
print('---')
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))
print('---')
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))


torch.Size([32, 65])
tensor(5.0364, grad_fn=<NllLossBackward0>)

l-QYjt'CL?jLDuQcLzy'RIo;'KdhpV
vLixa,nswYZwLEPS'ptIZqOZJ$CA$zy-QTkeMk x.gQSFCLg!iW3fO!3DGXAqTsq3pdgq
---

!LznIeJydZJSrFSrPLR!:VwWSmFNxbjPiNYQ:sry,OfKrxfvJI$WS3JqCbB-TSQXeKroeZfPL&,:opkl;Bvtz$LmOMyDjxxaZWtp
---

v,OxZQsWZalk'uxajqgoSXAWt'e.Q$.lE-aV
;spkRHcpkdot:u'-NGEzkMPy'hZCWhv.w.q!f'mOxF&IDRR,x
?$Ox?xj.BHJsG


Попытаемся это обучить!

In [48]:
@torch.no_grad()
def estimate_loss(model,eval_iters=100):
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [49]:

optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)
batch_size = 32
for _ in range(20):
    for steps in range(1000): # increase number of steps for good results...

        # sample a batch of data
        xb, yb = get_batch('train')

        # evaluate the loss
        logits, loss = m(xb, yb)
        optimizer.zero_grad(set_to_none=True)
        loss.backward()
        optimizer.step()

    print(estimate_loss(m))


{'train': tensor(3.6930), 'val': tensor(3.7040)}
{'train': tensor(3.1094), 'val': tensor(3.1279)}
{'train': tensor(2.8065), 'val': tensor(2.8111)}
{'train': tensor(2.6512), 'val': tensor(2.6535)}
{'train': tensor(2.5492), 'val': tensor(2.5751)}
{'train': tensor(2.5202), 'val': tensor(2.5350)}
{'train': tensor(2.5023), 'val': tensor(2.5223)}
{'train': tensor(2.4875), 'val': tensor(2.5029)}
{'train': tensor(2.4660), 'val': tensor(2.4986)}
{'train': tensor(2.4801), 'val': tensor(2.4931)}
{'train': tensor(2.4444), 'val': tensor(2.4905)}
{'train': tensor(2.4615), 'val': tensor(2.4723)}
{'train': tensor(2.4635), 'val': tensor(2.4907)}
{'train': tensor(2.4479), 'val': tensor(2.5041)}
{'train': tensor(2.4632), 'val': tensor(2.4970)}
{'train': tensor(2.4638), 'val': tensor(2.4915)}
{'train': tensor(2.4548), 'val': tensor(2.4958)}
{'train': tensor(2.4424), 'val': tensor(2.4946)}
{'train': tensor(2.4533), 'val': tensor(2.4827)}
{'train': tensor(2.4488), 'val': tensor(2.4845)}


In [50]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))
print('---')
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))
print('---')
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))


S faregur t ayorff houndfis cod sws h ovig nd fa his d R aclidipo her d
PSALedrs fureauplf tton mags
---

Tofee, feeay ous?
an stheh mis I maky,
Whery wanotheliusown anithy g, tom o whoull ou teedow tipino 
---

O, n tr No.
IO w sey t
Tho Cl s rteastthes mensithiraserenm, u semeyor at ng lcke tr semivengincasth


## Как сделать модель поумнее?

Сейчас мы пытаемся предсказать символ, глядя только на один предыдущий. Кажется, язык устроен чуть сложнее. Давайте попробуем как-нибудь учесть большее число символов нашей последовательности. Например, попробуем их усреднить.

Для начала научимся эффективно усреднять столбцы случайной матрицы:

In [51]:
torch.manual_seed(1337)
B,T,C = 4,8,2 # batch, time, channels
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [52]:
# не очень-то эффективно
xbow = torch.zeros((B,T,C)) # bow - потому что Bag Of Words. Помните усреднение векторов в Word2Vec, которое так называлось? Вот, термин оттуда.
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # (t,C)
        xbow[b,t] = torch.mean(xprev, 0)

In [53]:
# гораздо лучше
wei = torch.tril(torch.ones(T, T))     # нижнетреугольная матрица из единиц
wei = wei / wei.sum(1, keepdim=True)   # нормируем строки
xbow2 = wei @ x # (B, T, T) @ (B, T, C) ----> (B, T, C)

torch.allclose(xbow, xbow2, atol=1e-7) # результат правильный

True

In [54]:
# Пока непонятно зачем, но тоже работает
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x

torch.allclose(xbow, xbow3, atol=1e-7) # результат правильный


True

Сделаем из этого что-то полезное:

In [55]:
torch.manual_seed(1337)
B,T,C = 4,8,32 # batch, time, channels
x = torch.randn(B,T,C)

# Эта конструкция - то, что называется self-attention
head_size = 32
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x)   # (B, T, 16)
q = query(x) # (B, T, 16)
wei =  q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) ---> (B, T, T)

tril = torch.tril(torch.ones(T, T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)

v = value(x)
out = wei @ v

out.shape

torch.Size([4, 8, 32])

In [56]:
wei[0]

tensor([[1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00],
        [8.3841e-01, 1.6159e-01, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00],
        [8.8476e-01, 1.0445e-01, 1.0783e-02, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00],
        [9.5338e-02, 6.5157e-01, 1.8913e-01, 6.3963e-02, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00],
        [8.6153e-01, 2.7327e-02, 3.9553e-03, 1.6660e-02, 9.0524e-02, 0.0000e+00,
         0.0000e+00, 0.0000e+00],
        [4.6015e-05, 3.2064e-05, 1.1772e-02, 4.2589e-01, 2.5680e-06, 5.6226e-01,
         0.0000e+00, 0.0000e+00],
        [5.4061e-02, 6.4071e-02, 2.5209e-01, 8.7270e-02, 2.3189e-01, 2.3952e-01,
         7.1099e-02, 0.0000e+00],
        [5.4092e-02, 1.0762e-01, 1.6648e-02, 1.6889e-01, 6.2352e-01, 6.3775e-03,
         9.1127e-03, 1.3737e-02]], grad_fn=<SelectBackward0>)

Картинка [отсюда](https://sebastianraschka.com/blog/2023/self-attention-from-scratch.html)

<img src=images/summary.png width=1200></img>

С помощью этого механизма можно учитывать более важные куски с большим весом, а неважные - не учитывать вовсе. Давайте разбираться на примере картинки [отсюда](https://arxiv.org/pdf/1601.06733.pdf)

<img src=images/fbi.png width=800></img>

Важное

* Attention - это механизм для коммуникации. Это можно представить как узлы направленного графа, которые обращаются друг к другу и агрегируют информацию с использованием взвешенной суммы от всех связанных узлов, с весами, зависящими от данных.
* Никак не учитывается пространства, или порядок символов. Attention просто действует над набором векторов с помощью коммутативных операций. **С этим нужно что-то сделать, т.к. порядок символов для языка важен**
* Каждый пример в батче обрабатывается полностью независимо, и они никак не "общаются" друг с другом.

Попытаемся впилить это в нашу модель.

In [57]:
class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size, n_embd, context_size, dropout):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(context_size, context_size)))
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,head_size)
        q = self.query(x) # (B,T,head_size)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, head_size) @ (B, head_size, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei) #! применяем дропаут
        v = self.value(x) # (B,T,head_size)
        out = wei @ v # (B, T, T) @ (B, T, head_size) -> (B, T, head_size)
        return out

In [58]:
class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size, context_size, head_size, n_embd, dropout):
        super().__init__()
        self.context_size = context_size
        # теперь мы маппим наши символы в пространство размерности n_embd
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)

        # Добавим информацию о позициях символов
        self.position_embedding_table = nn.Embedding(context_size, n_embd)

        self.sa_head = Head(head_size=head_size, n_embd=n_embd,
                            context_size=context_size, dropout=dropout)
        self.lm_head = nn.Linear(head_size, vocab_size)


    def forward(self, idx, targets=None, device='cpu'):
        idx = idx.to(device)
        if targets is not None:
            targets = targets.to(device)
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (B,T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.sa_head(x) # (B,T,C)
        logits = self.lm_head(x)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # чтобы не переполнились позиционные эмбеддинги, подрежем контекст
            idx_cond = idx[:,-self.context_size:]
            # get the predictions
            logits, loss = self(idx_cond)
#!          смотрим только на предыдущий символ!
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
#!          берём не какой-то один самый вероятный символ, а сэмплируем случайный, используя полученные вероятности.
#!          в языке работает также есть [чёрный к...]от, а есть [чёрный к...]амаз
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

In [59]:

new_context_size = 8
model = BigramLanguageModel(vocab_size, context_size=new_context_size, head_size=32, n_embd=32, dropout=0.2)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)
batch_size = 32
for _ in range(10):
    for steps in range(2000): # increase number of steps for good results...

        # sample a batch of data
        xb, yb = get_batch('train',new_context_size)

        # evaluate the loss
        logits, loss = model.forward(xb, yb,device='cpu')
        optimizer.zero_grad(set_to_none=True)
        loss.backward()
        optimizer.step()

    print(estimate_loss(model))


{'train': tensor(2.4890), 'val': tensor(2.4778)}
{'train': tensor(2.4431), 'val': tensor(2.4458)}
{'train': tensor(2.4215), 'val': tensor(2.4262)}
{'train': tensor(2.4001), 'val': tensor(2.4238)}
{'train': tensor(2.3775), 'val': tensor(2.4081)}
{'train': tensor(2.3716), 'val': tensor(2.3919)}
{'train': tensor(2.3810), 'val': tensor(2.3907)}
{'train': tensor(2.3645), 'val': tensor(2.3885)}
{'train': tensor(2.3698), 'val': tensor(2.3812)}
{'train': tensor(2.3600), 'val': tensor(2.3813)}


In [60]:

print(decode(model.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))
print('---')
print(decode(model.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))
print('---')
print(decode(model.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))



Te,
Wipturdan yr ding,
Th hoendde ongt at
Alis I St lll ms thit h viloeve  ry--fpe nous torow imt bt
---

MEL ISCofr ouf, oun sot tito.
avemiegre e souse athiletnour, ncdaauscheru ke have, Sons ye me.

em
W
---


NGEort wdinr usuts
eriche ad azen BwTa b the;
o wn ogivindsw ds ne?T:

At; or fick, lsunleve ant la


<img src=images/transformer.png width=600></img>

* В encoder attention просто удаляется одна строка, которая выполняет маскирование с помощью tril, позволяя всем токенам общаться. Если маскирование есть, то блок называется decoder attention.
* "Self-attention" означает, что ключи и значения производятся из того же источника, что и запросы. В "перекрёстном внимании" запросы по-прежнему производятся из x, но ключи и значения поступают из какого-то другого, внешнего источника (например, из модуля кодировщика).
* "Scaled-attention" дополнительно делит wei на корень квадратный из размерности данных. Это делается для того, чтобы при единичной дисперсии входных Q, K, wei также имела единичную дисперсию, и функция Softmax оставалась распределённой, а не насыщалась слишком сильно, обращаясь в пик. Иллюстрация ниже.

In [61]:
# hyperparameters
batch_size = 64 # how many independent sequences will we process in parallel?
context_size = 256 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 500
learning_rate = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 384
n_head = 6
n_layer = 6
dropout = 0.2
# ------------

torch.manual_seed(1337)

##! new
class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size=head_size, context_size=context_size,n_embd=n_embd,dropout=dropout) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

##! new
class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

##! new
class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

class GPTLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(context_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

        # better init, not covered in the original GPT video, but important, will cover in followup video
        self.apply(self._init_weights)

    def _init_weights(self, module): #!! 
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        B, T = idx.shape
        idx = idx.to(device)
        if targets is not None:
            targets = targets.to(device)
        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last context_size tokens
            idx_cond = idx[:, -context_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = GPTLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss(m)
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")      
        context = torch.zeros((1, 1), dtype=torch.long, device=device)
        print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))
        print('-----')

    # sample a batch of data
    xb, yb = get_batch('train', context_size)

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))

10.788929 M parameters
step 0: train loss 4.2363, val loss 4.2397

SaVqulrf&&;gSYUCZo-TR,DMg
qlHHkSkrj!'VTSbmrMynJK.H3KU.nbO,h&MHKHP;
qkjno n$ReYPV$!DAyC Dbb$Ib&KIGEfr.Ca-k$orYHaorCZcn
owkvu'WP.LvDFxdFh,jI.DINfmSBQzkma
Ib,sO-
ErmaEkLcQoq'FsxYmshIbPU!-T'srnjwFsqhg.GpckkVSV,KD?R, z?Zyk.D:FB-C3raj.-$JJ-soQ!,$D;;p lfxSsN
LQm;b
ofDxto&GEN!:cEsnRC?,kHozPuYWJmiUMF;nIH,DxjGPgLwo.3Io$dAl!B?kUJS$KIrqDJzp
K!nxWWfYax ,PhtgCAMYn$;?3K!t,z3
NQF.yEauUDcZ m MSiFJl-$oFj$xldQDquDsig-.eevDi dm'gokmq$dft,BxmyVwo-r BHMjwI
Xyyro; PTumsy-cCUah dMA
MirDMATVx$yAkVxh$'w
x$!G3SHiM-Ts,&
sM
-----
step 500: train loss 2.1315, val loss 2.2017

MUTIUS:
GLode if ywill vandd unt an cimant lugdy,
Unow the then lupant In woal th thisthey!

MENENENIUS:
Costate unnd gad the ofor sasto 'sicie,
We in to he glio in no wix siced I ta o
Wickid durod be! I theee depege fonce for urth all
Alind thifer se, land tlame my fall mus lis;
Anove is ther dethaar ton o wilcon id,
I Pror oablach,
aK heme, ast gryef a t loman be sese!-leee c

# Ссылки

1. [Neural Networks: Zero to Hero](https://www.youtube.com/watch?v=VMj-3S1tku0&list=PLAqhIrjkxbuWI23v9cThsA9GvCAUhRvKZ&index=1&t=0s). Лучшее, что вы можете посмотреть, чтобы полноценно вкатиться нейросети и докатиться до понимания того, как под капотом работает ChatGPT
2. [Transformer, explained in detail](https://youtu.be/iOrNbK2T92M?feature=shared&t=18) хорошая русскоязычная лекция про трансформеры
3. Честное [объяснение](https://towardsdatascience.com/master-positional-encoding-part-i-63c05d90a0c3) тригонометрических позиционных энкодингов. Требуется VPN, если нет, то копия в pdf-формате лежит [рядом](Master%20Positional%20Encoding_%20Part%20I%20_%20by%20Jonathan%20Kernes%20_%20Towards%20Data%20Science.pdf)
4. [Имплементация GPT-2](https://github.com/karpathy/nanoGPT) в 300 строк питона
5. Хорошая статья про [Attention](https://sebastianraschka.com/blog/2023/self-attention-from-scratch.html)
6. [Аттеншен и трансформер в картинках](https://jalammar.github.io/illustrated-transformer/) 